In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode, lower, regexp_replace, collect_set, size, array_intersect
import socket
import time

# --- Configuração do Spark Session ---
local_ip = socket.gethostbyname(socket.gethostname())

spark = SparkSession.builder \
    .appName("Workload Pesado - Vocabulario Comum Entre Generos") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

print("Spark Session iniciada com sucesso!")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/09 05:40:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session iniciada com sucesso!


In [2]:
def executar_workload_sem_numpy():
    """
    Executa um workload pesado para encontrar o número de palavras em comum
    entre os principais gêneros musicais.
    """
    # --- 1. Carregar e Preparar os Dados ---
    caminho_parquet = "/spark-data/musicas_limpas.parquet"
    df = spark.read.parquet(caminho_parquet)

    # Filtra para os 10 gêneros mais populares para o teste
    top_genres = [row['main_genre'] for row in df.groupBy("main_genre").count().orderBy(col("count").desc()).limit(10).collect()]
    df_filtrado = df.filter(col("main_genre").isin(top_genres))

    # --- 2. Processamento e Agregação de Palavras por Gênero ---
    df_palavras = df_filtrado.withColumn("palavra", explode(split(lower(regexp_replace(col("text"), r'[\W_]+', ' ')), ' '))) \
                             .select("main_genre", "palavra") \
                             .filter(col("palavra") != "")

    df_vocabulario_genero = df_palavras.groupBy("main_genre").agg(collect_set("palavra").alias("vocabulario"))

    # --- 3. Self-Join e Cálculo da Interseção (A PARTE PESADA) ---
    df_g1 = df_vocabulario_genero.alias("g1")
    df_g2 = df_vocabulario_genero.alias("g2")

    df_pares_generos = df_g1.crossJoin(df_g2) \
                            .filter(col("g1.main_genre") < col("g2.main_genre"))

    # AQUI ESTÁ A CORREÇÃO: Usamos a função correta 'array_intersect'
    df_intersecao = df_pares_generos.withColumn(
        "palavras_em_comum",
        size(array_intersect(col("g1.vocabulario"), col("g2.vocabulario")))
    )

    # --- 4. Selecionar e Ordenar o Resultado Final ---
    df_resultado = df_intersecao.select(
        col("g1.main_genre").alias("genero_1"),
        col("g2.main_genre").alias("genero_2"),
        col("palavras_em_comum")
    ).orderBy(col("palavras_em_comum").desc())

    print(f"Workload concluído.")
    df_resultado.show(20, truncate=False)


# --- Medição do Tempo de Execução ---
print("Iniciando a execução do Workload Pesado (Sem Numpy)...")
inicio = time.time()

executar_workload_sem_numpy()

fim = time.time()
duracao = fim - inicio

print(f"\n--- Análise de Desempenho ---")
print(f"Tempo total de execução: {duracao:.2f} segundos")
print(f"----------------------------")

spark.stop()

Iniciando a execução do Workload Pesado (Sem Numpy)...


Workload concluído.


+----------------+--------+-----------------+
|genero_1        |genero_2|palavras_em_comum|
+----------------+--------+-----------------+
|hip hop         |rock    |74271            |
|hip hop         |rap     |53662            |
|hip hop         |pop     |53525            |
|pop             |rock    |42699            |
|rap             |rock    |42571            |
|hip hop         |jazz    |38935            |
|electronic      |hip hop |38474            |
|pop             |rap     |34925            |
|alternative rock|hip hop |34198            |
|electronic      |rock    |33560            |
|jazz            |rock    |33406            |
|folk            |hip hop |32331            |
|alternative rock|rock    |31153            |
|folk            |rock    |29437            |
|jazz            |pop     |29297            |
|electronic      |rap     |29284            |
|electronic      |pop     |29216            |
|jazz            |rap     |28869            |
|hip hop         |pop rock|28720  

In [3]:
spark.stop()